In [5]:
import pandas as pd
import numpy as np
import string
import Levenshtein as lev

# Punto 1. Limpieza y transformación de los datos

### 1. Carga de base datos

In [205]:
dataSource = "data/DataAnalyst_YoFio_PT_ocupaciones.csv"
data=pd.read_csv(dataSource)

### 2. Revisión de la consistencia de la base de datos

In [206]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Empleos  560 non-null    object
dtypes: object(1)
memory usage: 4.5+ KB


In [207]:
data=data.astype("string")

### 3. Normalización a minúsculas

In [208]:
data.Empleos=data.Empleos.apply(lambda x: x.lower())

### 4. Eliminación de puntuación y caracteres especiales

In [209]:
caracteresEspeciales=str.maketrans('áéíóú','aeiou',string.punctuation)
data.Empleos = data.Empleos.apply(lambda empleo: empleo.translate(caracteresEspeciales))

### 5. Separar por espacios

In [210]:
data.Empleos=data.Empleos.apply(str.split)

### 6. Eliminación de palabras con poco significado

In [211]:
preposiciones = ['a', 'ante', 'bajo', 'cabe', 'con', 'contra', 'de', 'desde', 'en', 'entre', 'hacia', 'hasta', 'para', 'por', 'segun', 'sin', 'so', 'sobre', 'tras', 'durante', 'mediante', 'versus','via']
conjunciones = ['y', 'e', 'ni', 'que', 'pero', 'mas', 'aunque', 'sino', 'siquiera', 'o', 'u', 'ora', 'sea', 'bien']
otras = ['soy', 'una', 'uno','la', 'el', 'los']
palabrasPocoSignificado=preposiciones + conjunciones + otras

In [212]:
def Eliminar_Palabras(listaDePalabras, palabrasParaEliminar=palabrasPocoSignificado):
    for indice, palabra in enumerate(listaDePalabras):
        if palabra in palabrasParaEliminar:
            listaDePalabras.pop(indice)

data.Empleos.apply(lambda empleos: Eliminar_Palabras(empleos));

### 7. Definición de grupos y reclasificación

In [213]:
grupos={'empleado': ['empleado','empleada','trabajador','obrero', 'tecnico', 'coordinador', 'operador', 'trabajo','supervisor',
                    'servidor', 'ayudante', 'auxiliar', 'mesero', 'promotor', 'despachador', 'demostrador',
                    'maniobrista','verificador','encargado','analista','sectretaria', 'montacarguista'
                    'instalador','oficina','fabrica','soporte','comisionista','montacarguista','maquilador','conserje',
                    'almacenista','telemarketer','telemarketing','callcenter','hoteleria','recepcionista',
                    'mantenimiento','intencencia','limpieza','aseo','reclutador','logistica','cuidador','reclutamiento',
                    'marketing','hostest','cafeteria','carpinteria','rosticeria','capturista','gasolinera','ajustador',
                    'autolavado','servicio','call','center','camarista','monirotista','aplicador','repartidor','paqueteria',
                    'cajero','mensajero','agente','checador','aseador','rh','rrhh','coordinacion'],
        'vendedor': ['vendedor', 'ventas', 'venta'], 
        'chofer':['chofer','uber', 'didi', 'taxi', 'taxsista', 'conductor','transportista','transporte','bicitaxi',
                 'trailero'],
        'salud': ['doctor','medico','enfermera','cirgujano','dentista','odontologo','camillero','radiologo',
                  'terapeuta','trasladista','clinico','preventista'],
        'comerciante':['comerciante','comercio', 'vendo'],
        'hogar':['hogar','ama', 'casa'],
        'oficio':['oficio','mecanico','electricista','albañil','vidriero','reposteria','carrocero','electrico', 'carnicero',
                 'hojalatero', 'cocinero', 'chef', 'zapatero','panadero','pizzero','estilista','maquillista',
                 'albañileria','herreria','tapicero','jardinero','costurero','sastre','construccion','repartidor',
                 'taller','pirotecnia','verdulero','estetica','fotografo','fotografia','manualidades','tatuador',
                 'reparacion','reparador','pastelero','repostero','afinador','banquetero','soldador','ebanista',
                 'entenador','automotriz','taquero'],
        'profesionista':['ingeniero', 'abogado','contador', 'licenciado','contabilidad', 'arquitecto', 'veterinario',
                        'metrologo','auditor','programador','contratista','mercadologo','profesionisnta','diseñador',
                        'diseño'],
        'ejecutivo':['ejecutivo','gerente', 'administrador', 'finanzas','jefe','administrativo','administracion'],
        'empresario':['empresario','tienda','negocio','locatario','empresa'],
        'seguridad':['seguridad', 'guardia', 'policia','custodia','vigilante','velador'],
        'jubilado':['jubilado', 'pensionado'],
        'independiente':['independiente','freelance','emprendedor'],
        'docente':['docente','profesor','maestro', 'instructor'],
        'consultor':['consultor', 'asesor'],
        'desempleado':['desempleado']
       }

In [214]:
def Clasificador_Empleo(listaPalabras, diccionarioGrupos=grupos, precision=3):
    for palabra in listaPalabras:
        for grupo in list(diccionarioGrupos.keys()):
            for elemento in diccionarioGrupos[grupo]:
                distancia=lev.distance(palabra,elemento)
                if distancia < precision:
                    return grupo
    return 'otro' #str(listaPalabras)

In [215]:
data['EmpleoClasificado']=''
data.EmpleoClasificado=data.Empleos.apply(lambda empleo: Clasificador_Empleo(empleo))
data=data.drop(['Empleos'], axis =1)
data.rename(columns={'EmpleoClasificado':'Empleos'})

### 8. Guardado de base de datos en un archivo

In [218]:
data.to_csv('data/Resultado_Clasificacion_ocupaciones.csv')

# Código para ejecución completa del Punto 1

In [220]:
import pandas as pd
import numpy as np
import string
import Levenshtein as lev

# Funciones

def Eliminar_Palabras(listaDePalabras, palabrasParaEliminar=palabrasPocoSignificado):
    for indice, palabra in enumerate(listaDePalabras):
        if palabra in palabrasParaEliminar:
            listaDePalabras.pop(indice)
            

def Clasificador_Empleo(listaPalabras, diccionarioGrupos=grupos, precision=3):
    for palabra in listaPalabras:
        for grupo in list(diccionarioGrupos.keys()):
            for elemento in diccionarioGrupos[grupo]:
                distancia=lev.distance(palabra,elemento)
                if distancia < precision:
                    return grupo
    return 'otro'

# Main
# 1. Carga de base datos
dataSource = "data/DataAnalyst_YoFio_PT_ocupaciones.csv"
data=pd.read_csv(dataSource)
# 2. Revisión de la consistencia de la base de datos
data=data.astype("string")
# 3. Normalización a minúsculas
data.Empleos=data.Empleos.apply(lambda x: x.lower())
# 4. Eliminación de puntuación y caracteres especiales
caracteresEspeciales=str.maketrans('áéíóú','aeiou',string.punctuation)
data.Empleos = data.Empleos.apply(lambda empleo: empleo.translate(caracteresEspeciales))
# 5. Separar por espacios
data.Empleos=data.Empleos.apply(str.split)
# 6. Eliminación de palabras con poco significado
preposiciones = ['a', 'ante', 'bajo', 'cabe', 'con', 'contra', 'de', 'desde', 'en', 'entre', 'hacia', 'hasta', 'para', 'por', 'segun', 'sin', 'so', 'sobre', 'tras', 'durante', 'mediante', 'versus','via']
conjunciones = ['y', 'e', 'ni', 'que', 'pero', 'mas', 'aunque', 'sino', 'siquiera', 'o', 'u', 'ora', 'sea', 'bien']
otras = ['soy', 'una', 'uno','la', 'el', 'los']
palabrasPocoSignificado=preposiciones + conjunciones + otras
data.Empleos.apply(lambda empleos: Eliminar_Palabras(empleos));
# 7. Definición de grupos y reclasificación
grupos={'empleado': ['empleado','empleada','trabajador','obrero', 'tecnico', 'coordinador', 'operador', 'trabajo','supervisor',
                    'servidor', 'ayudante', 'auxiliar', 'mesero', 'promotor', 'despachador', 'demostrador',
                    'maniobrista','verificador','encargado','analista','sectretaria', 'montacarguista'
                    'instalador','oficina','fabrica','soporte','comisionista','montacarguista','maquilador','conserje',
                    'almacenista','telemarketer','telemarketing','callcenter','hoteleria','recepcionista',
                    'mantenimiento','intencencia','limpieza','aseo','reclutador','logistica','cuidador','reclutamiento',
                    'marketing','hostest','cafeteria','carpinteria','rosticeria','capturista','gasolinera','ajustador',
                    'autolavado','servicio','call','center','camarista','monirotista','aplicador','repartidor','paqueteria',
                    'cajero','mensajero','agente','checador','aseador','rh','rrhh','coordinacion'],
        'vendedor': ['vendedor', 'ventas', 'venta'], 
        'chofer':['chofer','uber', 'didi', 'taxi', 'taxsista', 'conductor','transportista','transporte','bicitaxi',
                 'trailero'],
        'salud': ['doctor','medico','enfermera','cirgujano','dentista','odontologo','camillero','radiologo',
                  'terapeuta','trasladista','clinico','preventista'],
        'comerciante':['comerciante','comercio', 'vendo'],
        'hogar':['hogar','ama', 'casa'],
        'oficio':['oficio','mecanico','electricista','albañil','vidriero','reposteria','carrocero','electrico', 'carnicero',
                 'hojalatero', 'cocinero', 'chef', 'zapatero','panadero','pizzero','estilista','maquillista',
                 'albañileria','herreria','tapicero','jardinero','costurero','sastre','construccion','repartidor',
                 'taller','pirotecnia','verdulero','estetica','fotografo','fotografia','manualidades','tatuador',
                 'reparacion','reparador','pastelero','repostero','afinador','banquetero','soldador','ebanista',
                 'entenador','automotriz','taquero'],
        'profesionista':['ingeniero', 'abogado','contador', 'licenciado','contabilidad', 'arquitecto', 'veterinario',
                        'metrologo','auditor','programador','contratista','mercadologo','profesionisnta','diseñador',
                        'diseño'],
        'ejecutivo':['ejecutivo','gerente', 'administrador', 'finanzas','jefe','administrativo','administracion'],
        'empresario':['empresario','tienda','negocio','locatario','empresa'],
        'seguridad':['seguridad', 'guardia', 'policia','custodia','vigilante','velador'],
        'jubilado':['jubilado', 'pensionado'],
        'independiente':['independiente','freelance','emprendedor'],
        'docente':['docente','profesor','maestro', 'instructor'],
        'consultor':['consultor', 'asesor'],
        'desempleado':['desempleado']
       }
data['EmpleoClasificado']=''
data.EmpleoClasificado=data.Empleos.apply(lambda empleo: Clasificador_Empleo(empleo))
data=data.drop(['Empleos'], axis =1)
data.rename(columns={'EmpleoClasificado':'Empleos'})
# 8. Guardado de base de datos en un archivo
data.to_csv('data/Resultado_Clasificacion_ocupaciones.csv')


# Punto 3. Métricas a partir de los datos

### 1. Carga de base datos

In [578]:
totalProcesos=4
dataProcesos=[]
for numeroProceso in range(totalProcesos):
    dataProcesos.append(pd.read_csv("data/proceso"+ str(numeroProceso + 1) +".csv"))

### 2. Limpieza y Transformación

In [579]:
#Limpieza de datos nulos en campo address
dataProcesos[1].address=dataProcesos[1].address.apply(lambda x:'' if type(x)!=str else x)

#Validación Datos completos
#Registros con campos completos
listaRegistrosCompletos=list(dataProcesos[1][dataProcesos[1].address != ''].user_id)
#Validacion compra mayor a 40 pesos
#Registros con compraS iniciales mayores a 40 pesos
listaRegistrosMas40=list(dataProcesos[2][dataProcesos[2].tx1_amount>40].user_id)
#Combinacion de Validaciones
listaUsuariosValidos=[] #Contiene todos los user_id validos segun las restricciones dadas
for i in listaRegistrosCompletos:
    if i in listaRegistrosMas40:
        listaUsuariosValidos.append(i)
        
#Transformacion de srt a timestamp en las fechas de registro y eliminacion de la hora
from datetime import datetime
from datetime import timedelta

dataProcesos[0].registration_date=dataProcesos[0].registration_date.apply(lambda x:
                                  datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())

#Definicion del intervalo de tiempo
fechaInicial=dataProcesos[0].registration_date.min()
fechaFinal=dataProcesos[0].registration_date.max()
diasTotales=(fechaFinal-fechaInicial).days

### 3. Cálculo del First Payment Default Rate por dia

In [580]:
firstPaymentDefaultRate=[]
for i in range(diasTotales+1):
    #Generacion de listas de usuarios registrados en cada dia
    listaUsuariosHoy=list(dataProcesos[0][dataProcesos[0].registration_date == fechaInicial+timedelta(days=i)].user_id)
    totalCuentas=len(listaUsuariosHoy)
    nCuentasImpago=0
    
    for j in range(len(listaUsuariosHoy)):
        #Extraccion de valores de procesos 3 y 4 de los usuarios validados
        try:
            tx1_amount=float(dataProcesos[2][dataProcesos[2].user_id==listaUsuariosHoy[j]].tx1_amount)
        except:
            tx1_amount= 0.00
            
        try:
            payment_amount=float(dataProcesos[3][dataProcesos[3].user_id==listaUsuariosHoy[j]].payment_amount)
        except:
            payment_amount=0.00
            
        if tx1_amount-payment_amount>0:
            nCuentasImpago=+1
    
    firstPaymentDefaultRate.append([fechaInicial+timedelta(days=i),totalCuentas,nCuentasImpago,(nCuentasImpago/totalCuentas)*100])
        

In [581]:
dffPDR=pd.DataFrame(firstPaymentDefaultRate)
dffPDR.rename(columns={0:'Fecha',1:'Total_de_cuentas',2:'Cuentas_en_impago',3:'Porcentaje_cuentas_en_impago'}, inplace=True)

In [582]:
dffPDR.head(50)

,Fecha,Total_de_cuentas,Cuentas_en_impago,Porcentaje_cuentas_en_impago
0,2020-07-07,6,1,16.666667
1,2020-07-08,12,1,8.333333
2,2020-07-09,13,1,7.692308
3,2020-07-10,11,1,9.090909
4,2020-07-11,5,1,20.000000
5,2020-07-12,9,1,11.111111
6,2020-07-13,10,1,10.000000
7,2020-07-14,8,0,0.000000
8,2020-07-15,3,1,33.333333
9,2020-07-16,15,1,6.666667


In [577]:
dataProcesos[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   user_id     607 non-null    object 
 1   tx1_date    607 non-null    object 
 2   tx1_amount  607 non-null    float64
 3   tx2_date    607 non-null    object 
 4   tx2_amount  607 non-null    float64
dtypes: float64(2), object(3)
memory usage: 23.8+ KB
